## Steps for Building Network
[Step 1](#step1): Unzip image dataset and check out train, validation and test files.

[Step 2](#step2): Show image and json files in train & validation dataset.

[Step 3](#step3): Initialize features(input) and labels(output) from images and json list.

- read images from train/validation/test path.
- read labels from train/validation json file.
- resize and normalize images.
- get batch and return feature_batch and label_batch.

[Step 4](#step4): Build convolutional network, return training accuracy and training loss.

[Step 5](#step5): Train on steps = 20000.

[Step 6](#step6): Train on full dataset.
- epoch x, batch x, training loss, validation accuracy, evluation accuracy

[Step 7](#step7): Test and write json submit file.

<a id='step1'></a>
## Step 1: Unzip image dataset and check out train, validation and test files.

In [1]:
import os, zipfile

train_path = 'E:/ai_challenger/scene classification/dataset/ai_challenger_scene_train_20170904.zip'
validation_path = 'E:/ai_challenger/scene classification/dataset/ai_challenger_scene_validation_20170908.zip'
test_a_path = 'E:/ai_challenger/scene classification/dataset/ai_challenger_scene_test_a_20170922.zip'
extract_path = 'E:/ai_challenger/scene classification/dataset'

def unzip(zipfile_path, extract_path, zipfile_name):
    zipfile = zipfile.ZipFile(zipfile_path, 'r')
    print('Extracting {} ...'.format(zipfile_name))
    zipfile.extractall()
    zipfile.close()
    print('{} has been extracted.'.format(zipfle_name))

if os.path.exists(extract_path):
    print('Found extraced dataset')
else:
    unzip(train_path, extract_path, 'training dataset')
    unzip(validation_path, extract_path, 'validation dataset')
    unzip(test_a_path, extract_path, 'test dataset')

Found extraced dataset


<a id='step2'></a>
## Step 2: Show image and json files in train & validation dataset.

In [2]:
import json
import glob
from scipy.misc import imread, imresize, imsave
import numpy as np

train_features_path = r'E:\ai_challenger\scene classification\dataset\ai_challenger_scene_train_20170904\scene_train_images_20170904'
train_labels_path = r'E:\ai_challenger\scene classification\dataset\ai_challenger_scene_train_20170904\scene_train_annotations_20170904.json'
validation_features_path =r'E:\ai_challenger\scene classification\dataset\ai_challenger_scene_validation_20170908\scene_validation_images_20170908'
validation_labels_path =r'E:\ai_challenger\scene classification\dataset\ai_challenger_scene_validation_20170908\scene_validation_images_20170908\scene_validation_annotations_20170908.json'
test_a_features_path = r'E:\ai_challenger\scene classification\dataset\ai_challenger_scene_test_a_20170922\scene_test_a_images_20170922'

# Show train label list
with open(train_labels_path, 'r') as f:
    train_label_list = json.load(f)
    print(train_label_list[:10])
    train_dict = {}
    for image in train_label_list:
        train_dict[image['image_id']] = int(image['label_id'])
    print('\n')
    print(len(train_dict))     

# Show train features list resulting out of memory...

[{'label_id': '66', 'image_id': '79f993ae0858ae238b22968c5934d1ddba585ae4.jpg', 'image_url': 'https://n1-q.mafengwo.net/s1/M00/6B/72/wKgBm04Wc5WzFXU0AAHf09bdpiY84.jpeg?imageView2%2F2%2Fw%2F600%2Fq%2F90'}, {'label_id': '61', 'image_id': 'e963208fe9e90df0c385f7367bcdb6d0d5d0b165.jpg', 'image_url': 'http://news.sogou.com/'}, {'label_id': '64', 'image_id': '02df5ecbf7c749ccc9d833f129bbd5d9837940ce.jpg', 'image_url': 'http://img2.fawan.com/2016/12/30/e967f93e7713c57cd2b00b832dd6091a_500x-_90.jpg'}, {'label_id': '31', 'image_id': '5620eb385b7567fb087813cf5233b5ceecdeeca3.jpg', 'image_url': 'https://b1-q.mafengwo.net/s1/M00/F2/C9/wKgBm04Wx3a-gk2FAAKbPKX7E9w91.jpeg?imageView2%2F2%2Fw%2F600%2Fq%2F90'}, {'label_id': '19', 'image_id': 'f8b4d42001a562fc63b9b39c02531661c0e236ca.jpg', 'image_url': 'http://news.sogou.com/'}, {'label_id': '11', 'image_id': '57e7eb438670a4519041dab1482f2594a92f8a09.jpg', 'image_url': 'http://www.user2.jqw.com/2014/01/06/1347666/product/b201401072000291460.JPG'}, {'labe

<a id='step3'></a>
## Step 3: Initialize features(input) and labels(output) from images and json list.

In [3]:
import json
from scipy.misc import imread, imresize
import numpy as np
import os

class initialize(object):
    # Get image-label list for train and validation
    def __init__(self, feature_path, label_path):
        self.image_label_dict = {}
        with open(label_path, 'r') as f:
            label_list = json.load(f)
        for image in label_list:
            self.image_label_dict[image['image_id']] = int(image['label_id'])
        self.start = 0
        self.end = 0
        self.length = len(self.image_label_dict) # number of feature images
        self.image_name = list(self.image_label_dict.keys())
        self.feature_path = feature_path
    
    # Read image in feature path, resize and normalize to [-1, 1]
    def get_image(self, image_path, image_size):
        image = imread(image_path)
        image = imresize(image, [image_size, image_size])       
        image = np.array(image).astype(np.float32)
        image = 2 * (image - np.min(image)) / np.ptp(image) - 1
        return image
    
    # Get feature and label batch
    def get_batch(self, batch_size, image_size):
        self.start = self.end
        if self.start >= self.length:
            self.start = 0
        batch_feature = []
        batch_label = []
        index = self.start
        while len(batch_feature) < batch_size:
            if index >= self.length:
                index = 0
            i_image_path = os.path.join(self.feature_path, self.image_name[index])
            i_image = self.get_image(i_image_path, image_size)
            i_label = self.image_label_dict[self.image_name[index]]
            batch_feature.append(i_image)
            batch_label.append(i_label)
            index += 1
        self.end = index
        return batch_feature, batch_label

<a id='step4'></a>
## Step 4: 16 convolutional layers with batch normalization.

In [4]:
import tensorflow as tf

def conv_layer(input_layer, filters, strides, is_training):
    layer = tf.layers.conv2d(
        inputs=input_layer, 
        filters=filters, 
        kernel_size=3,
        strides=strides, 
        padding='same', 
        activation=None,
        kernel_initializer=tf.truncated_normal_initializer()
    )
    
    layer = tf.layers.batch_normalization(
        inputs=layer, 
        axis=-1,
        momentum=0.9,
        epsilon=0.001,
        center=True,
        scale=True,
        training=is_training
    )
    
    layer = tf.nn.relu(layer)
    return layer

def fully_connected(input_layer, num_units, is_training):
    layer = tf.layers.dense(input_layer, num_units, use_bias=False, activation=None)
    layer = tf.layers.batch_normalization(layer, training=is_training)
    layer = tf.nn.relu(layer)
    return layer

def conv_network(feature, label, num_class, image_size, keep_prob, is_training):
    input_layer = tf.reshape(feature, [-1, image_size, image_size, 3])
    
    # 16 conv layers with 64, 128, 256, 512 filters.
    layer = conv_layer(input_layer, 64, 1, is_training)
    layer = conv_layer(layer, 64, 1, is_training)
    layer = tf.layers.max_pooling2d(layer, pool_size=[2, 2], strides=2, padding='same')
    
    layer = conv_layer(layer, 128, 1, is_training)
    layer = conv_layer(layer, 128, 1, is_training)
    layer = tf.layers.max_pooling2d(layer, pool_size=[2, 2], strides=2, padding='same')
    
    layer = conv_layer(layer, 256, 2, is_training)
    layer = conv_layer(layer, 256, 2, is_training)
    layer = conv_layer(layer, 256, 2, is_training)
    layer = conv_layer(layer, 256, 2, is_training)
    layer = tf.layers.max_pooling2d(layer, pool_size=[2, 2], strides=2, padding='same')
    
    layer = conv_layer(layer, 512, 2, is_training)
    layer = conv_layer(layer, 512, 2, is_training)
    layer = conv_layer(layer, 512, 2, is_training)
    layer = conv_layer(layer, 512, 2, is_training)
    layer = tf.layers.max_pooling2d(layer, pool_size=[2, 2], strides=2, padding='same')
    
    layer = conv_layer(layer, 512, 2, is_training)
    layer = conv_layer(layer, 512, 2, is_training)
    layer = conv_layer(layer, 512, 2, is_training)
    layer = conv_layer(layer, 512, 2, is_training)
    layer = tf.layers.max_pooling2d(layer, pool_size=[2, 2], strides=2, padding='same')
    
    shape = layer.get_shape().as_list()
    layer = tf.reshape(layer, shape=[-1, shape[1]*shape[2]*shape[3]])
    layer = fully_connected(layer, 800, is_training)
    layer = tf.nn.dropout(layer, keep_prob)
    logits = tf.layers.dense(layer, 80)
    output = tf.sigmoid(logits)
    
    # Loss and optimizer
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=label))
    
    return logits, loss


In [5]:
def train(train_feature_path, train_label_path, checkpoint_path, num_class, batch_size, image_size, max_step):
    
    train = initialize(train_feature_path, train_label_path)
        
    feature = tf.placeholder(tf.float32, shape=[None, image_size, image_size, 3], name='feature')
    label = tf.placeholder(tf.float32, shape=[None], name='label')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    one_hot_label = tf.one_hot(indices=tf.cast(label, tf.int32), depth=80)
    is_training = tf.placeholder(tf.bool, name='is_training')
    
    logits, loss = conv_network(feature, one_hot_label, num_class, image_size, keep_prob, is_training)
    
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
        train_opt = tf.train.AdamOptimizer(learning_rate).minimize(loss)
        
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_label, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
       
    with tf.Session() as sess:
        saver = tf.train.Saver()
        ckpt = tf.train.get_checkpoint_state(checkpoint_path)
        
        if ckpt and ckpt.model_checkpoint_path:
            print('Restore the model from checkpoint {}.'.format(ckpt.model_checkpoint_path))
            start_step = int(ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1])
        else:
            sess.run(tf.global_variables_initializer())
            start_step = 0
            print('Start training from new start.')
        
        for steps in range(start_step, start_step + max_step):
            train_feature_batch, train_label_batch = train.get_batch(batch_size, image_size)
            #print(is_training, feature, label)
            sess.run(train_opt, feed_dict={feature: train_feature_batch, label: train_label_batch, keep_prob: 0.5, is_training: True})
                
            if steps % 10 == 0:
                train_accuracy = sess.run(accuracy, feed_dict={feature: train_feature_batch, label: train_label_batch, keep_prob: 0.5, is_training: False})
                train_loss = sess.run(loss, feed_dict={feature: train_feature_batch, label: train_label_batch, keep_prob: 0.5, is_training: False})
                print('Step {}'.format(steps),
                      'Training Accuracy {:.3f}...'.format(train_accuracy),
                      'Training Loss {:.3f}...'.format(train_loss),
                     ) 
            if steps % 1000 == 0:
                saver.save(sess, checkfile, global_step=steps)
                print('Writing checkpoint at step {}'.format(steps))

        print('Training completed.')

# image pathes
train_feature_path = r'E:\ai_challenger\scene classification\dataset\ai_challenger_scene_train_20170904\scene_train_images_20170904'
train_label_path = r'E:\ai_challenger\scene classification\dataset\ai_challenger_scene_train_20170904\scene_train_annotations_20170904.json'
checkpoint_path = './checkpoint/'
checkfile = './checkpoint/model.ckpt'

num_class = 80
batch_size = 64
image_size = 64
max_step = 60000
learning_rate =0.002

train(train_feature_path, train_label_path, checkpoint_path, num_class, batch_size, image_size, max_step)

Start training from new start.
Step 0 Training Accuracy 0.047... Training Loss 989718720.000...
Writing checkpoint at step 0
Step 10 Training Accuracy 0.016... Training Loss 456.297...
Step 20 Training Accuracy 0.016... Training Loss 22.131...
Step 30 Training Accuracy 0.000... Training Loss 7.393...
Step 40 Training Accuracy 0.000... Training Loss 5.078...
Step 50 Training Accuracy 0.016... Training Loss 4.716...
Step 60 Training Accuracy 0.047... Training Loss 5.275...
Step 70 Training Accuracy 0.016... Training Loss 4.825...
Step 80 Training Accuracy 0.047... Training Loss 4.834...
Step 90 Training Accuracy 0.016... Training Loss 4.944...
Step 100 Training Accuracy 0.031... Training Loss 5.004...
Step 110 Training Accuracy 0.031... Training Loss 4.616...
Step 120 Training Accuracy 0.047... Training Loss 5.456...
Step 130 Training Accuracy 0.016... Training Loss 5.189...
Step 140 Training Accuracy 0.000... Training Loss 4.922...
Step 150 Training Accuracy 0.000... Training Loss 4.803

Step 1370 Training Accuracy 0.188... Training Loss 3.108...
Step 1380 Training Accuracy 0.125... Training Loss 3.513...
Step 1390 Training Accuracy 0.172... Training Loss 3.225...
Step 1400 Training Accuracy 0.125... Training Loss 3.338...
Step 1410 Training Accuracy 0.078... Training Loss 3.756...
Step 1420 Training Accuracy 0.109... Training Loss 3.407...
Step 1430 Training Accuracy 0.219... Training Loss 3.397...
Step 1440 Training Accuracy 0.203... Training Loss 3.245...
Step 1450 Training Accuracy 0.203... Training Loss 3.260...
Step 1460 Training Accuracy 0.234... Training Loss 3.130...
Step 1470 Training Accuracy 0.141... Training Loss 3.099...
Step 1480 Training Accuracy 0.141... Training Loss 3.527...
Step 1490 Training Accuracy 0.172... Training Loss 3.617...
Step 1500 Training Accuracy 0.172... Training Loss 3.376...
Step 1510 Training Accuracy 0.172... Training Loss 3.254...
Step 1520 Training Accuracy 0.219... Training Loss 3.102...
Step 1530 Training Accuracy 0.125... Tra

Step 2740 Training Accuracy 0.188... Training Loss 3.120...
Step 2750 Training Accuracy 0.172... Training Loss 3.282...
Step 2760 Training Accuracy 0.328... Training Loss 2.552...
Step 2770 Training Accuracy 0.219... Training Loss 2.770...
Step 2780 Training Accuracy 0.266... Training Loss 2.837...
Step 2790 Training Accuracy 0.141... Training Loss 3.078...
Step 2800 Training Accuracy 0.391... Training Loss 2.527...
Step 2810 Training Accuracy 0.234... Training Loss 2.778...
Step 2820 Training Accuracy 0.266... Training Loss 2.931...
Step 2830 Training Accuracy 0.234... Training Loss 3.101...
Step 2840 Training Accuracy 0.312... Training Loss 2.725...
Step 2850 Training Accuracy 0.266... Training Loss 2.817...
Step 2860 Training Accuracy 0.234... Training Loss 2.905...
Step 2870 Training Accuracy 0.234... Training Loss 2.706...
Step 2880 Training Accuracy 0.250... Training Loss 2.804...
Step 2890 Training Accuracy 0.344... Training Loss 2.793...
Step 2900 Training Accuracy 0.172... Tra

Step 4100 Training Accuracy 0.344... Training Loss 2.518...
Step 4110 Training Accuracy 0.266... Training Loss 2.599...
Step 4120 Training Accuracy 0.297... Training Loss 2.797...
Step 4130 Training Accuracy 0.234... Training Loss 3.052...
Step 4140 Training Accuracy 0.266... Training Loss 2.611...
Step 4150 Training Accuracy 0.344... Training Loss 2.611...
Step 4160 Training Accuracy 0.328... Training Loss 2.553...
Step 4170 Training Accuracy 0.234... Training Loss 3.171...
Step 4180 Training Accuracy 0.250... Training Loss 3.054...
Step 4190 Training Accuracy 0.312... Training Loss 2.773...
Step 4200 Training Accuracy 0.281... Training Loss 2.734...
Step 4210 Training Accuracy 0.375... Training Loss 2.548...
Step 4220 Training Accuracy 0.344... Training Loss 2.457...
Step 4230 Training Accuracy 0.266... Training Loss 2.464...
Step 4240 Training Accuracy 0.438... Training Loss 2.294...
Step 4250 Training Accuracy 0.406... Training Loss 2.551...
Step 4260 Training Accuracy 0.250... Tra

Step 5470 Training Accuracy 0.359... Training Loss 2.247...
Step 5480 Training Accuracy 0.359... Training Loss 2.507...
Step 5490 Training Accuracy 0.406... Training Loss 2.189...
Step 5500 Training Accuracy 0.234... Training Loss 2.705...
Step 5510 Training Accuracy 0.375... Training Loss 2.694...
Step 5520 Training Accuracy 0.359... Training Loss 2.358...
Step 5530 Training Accuracy 0.297... Training Loss 2.255...
Step 5540 Training Accuracy 0.391... Training Loss 2.206...
Step 5550 Training Accuracy 0.578... Training Loss 1.731...
Step 5560 Training Accuracy 0.219... Training Loss 2.822...
Step 5570 Training Accuracy 0.344... Training Loss 2.288...
Step 5580 Training Accuracy 0.344... Training Loss 2.152...
Step 5590 Training Accuracy 0.359... Training Loss 2.434...
Step 5600 Training Accuracy 0.422... Training Loss 2.275...
Step 5610 Training Accuracy 0.312... Training Loss 2.305...
Step 5620 Training Accuracy 0.391... Training Loss 2.529...
Step 5630 Training Accuracy 0.422... Tra

...

Step 53800 Training Accuracy 0.953... Training Loss 0.103...
Step 53810 Training Accuracy 0.984... Training Loss 0.120...
Step 53820 Training Accuracy 0.969... Training Loss 0.128...
Step 53830 Training Accuracy 1.000... Training Loss 0.060...
Step 53840 Training Accuracy 0.969... Training Loss 0.130...
Step 53850 Training Accuracy 0.953... Training Loss 0.151...
Step 53860 Training Accuracy 0.953... Training Loss 0.165...
Step 53870 Training Accuracy 0.953... Training Loss 0.098...
Step 53880 Training Accuracy 0.938... Training Loss 0.126...
Step 53890 Training Accuracy 0.969... Training Loss 0.128...
Step 53900 Training Accuracy 0.875... Training Loss 0.432...
Step 53910 Training Accuracy 1.000... Training Loss 0.082...
Step 53920 Training Accuracy 0.953... Training Loss 0.166...
Step 53930 Training Accuracy 0.969... Training Loss 0.092...
Step 53940 Training Accuracy 1.000... Training Loss 0.043...
Step 53950 Training Accuracy 0.984... Training Loss 0.111...
Step 53960 Training Accu

Step 55140 Training Accuracy 0.938... Training Loss 0.191...
Step 55150 Training Accuracy 1.000... Training Loss 0.052...
Step 55160 Training Accuracy 1.000... Training Loss 0.023...
Step 55170 Training Accuracy 0.969... Training Loss 0.054...
Step 55180 Training Accuracy 0.969... Training Loss 0.111...
Step 55190 Training Accuracy 0.984... Training Loss 0.081...
Step 55200 Training Accuracy 0.969... Training Loss 0.130...
Step 55210 Training Accuracy 0.969... Training Loss 0.164...
Step 55220 Training Accuracy 1.000... Training Loss 0.035...
Step 55230 Training Accuracy 0.953... Training Loss 0.057...
Step 55240 Training Accuracy 1.000... Training Loss 0.053...
Step 55250 Training Accuracy 0.922... Training Loss 0.215...
Step 55260 Training Accuracy 0.969... Training Loss 0.046...
Step 55270 Training Accuracy 0.953... Training Loss 0.093...
Step 55280 Training Accuracy 0.938... Training Loss 0.356...
Step 55290 Training Accuracy 0.984... Training Loss 0.092...
Step 55300 Training Accu

Step 56480 Training Accuracy 0.969... Training Loss 0.071...
Step 56490 Training Accuracy 0.984... Training Loss 0.071...
Step 56500 Training Accuracy 0.922... Training Loss 0.166...
Step 56510 Training Accuracy 0.938... Training Loss 0.101...
Step 56520 Training Accuracy 0.969... Training Loss 0.097...
Step 56530 Training Accuracy 0.922... Training Loss 0.166...
Step 56540 Training Accuracy 0.953... Training Loss 0.064...
Step 56550 Training Accuracy 0.938... Training Loss 0.100...
Step 56560 Training Accuracy 0.953... Training Loss 0.125...
Step 56570 Training Accuracy 0.969... Training Loss 0.109...
Step 56580 Training Accuracy 1.000... Training Loss 0.064...
Step 56590 Training Accuracy 0.984... Training Loss 0.084...
Step 56600 Training Accuracy 0.984... Training Loss 0.024...
Step 56610 Training Accuracy 0.953... Training Loss 0.144...
Step 56620 Training Accuracy 0.953... Training Loss 0.044...
Step 56630 Training Accuracy 0.984... Training Loss 0.055...
Step 56640 Training Accu

Step 57820 Training Accuracy 0.969... Training Loss 0.159...
Step 57830 Training Accuracy 0.938... Training Loss 0.116...
Step 57840 Training Accuracy 0.984... Training Loss 0.108...
Step 57850 Training Accuracy 0.922... Training Loss 0.146...
Step 57860 Training Accuracy 1.000... Training Loss 0.042...
Step 57870 Training Accuracy 0.953... Training Loss 0.045...
Step 57880 Training Accuracy 0.953... Training Loss 0.114...
Step 57890 Training Accuracy 0.984... Training Loss 0.063...
Step 57900 Training Accuracy 0.984... Training Loss 0.077...
Step 57910 Training Accuracy 0.953... Training Loss 0.114...
Step 57920 Training Accuracy 0.922... Training Loss 0.138...
Step 57930 Training Accuracy 1.000... Training Loss 0.039...
Step 57940 Training Accuracy 0.938... Training Loss 0.082...
Step 57950 Training Accuracy 0.969... Training Loss 0.100...
Step 57960 Training Accuracy 0.938... Training Loss 0.087...
Step 57970 Training Accuracy 0.969... Training Loss 0.078...
Step 57980 Training Accu

Step 59160 Training Accuracy 0.953... Training Loss 0.214...
Step 59170 Training Accuracy 0.984... Training Loss 0.109...
Step 59180 Training Accuracy 0.984... Training Loss 0.046...
Step 59190 Training Accuracy 0.953... Training Loss 0.111...
Step 59200 Training Accuracy 0.953... Training Loss 0.297...
Step 59210 Training Accuracy 0.953... Training Loss 0.119...
Step 59220 Training Accuracy 0.938... Training Loss 0.096...
Step 59230 Training Accuracy 1.000... Training Loss 0.026...
Step 59240 Training Accuracy 0.938... Training Loss 0.184...
Step 59250 Training Accuracy 0.969... Training Loss 0.090...
Step 59260 Training Accuracy 0.984... Training Loss 0.068...
Step 59270 Training Accuracy 0.969... Training Loss 0.084...
Step 59280 Training Accuracy 0.984... Training Loss 0.032...
Step 59290 Training Accuracy 0.906... Training Loss 0.247...
Step 59300 Training Accuracy 0.984... Training Loss 0.089...
Step 59310 Training Accuracy 0.953... Training Loss 0.128...
Step 59320 Training Accu

<a id='step5'></a>
## Step 5: Test model and write result into submit.json(image id, top 3 label id)

In [9]:
def test(val_feature_path, val_label_path, checkpoint_dir, num_class, image_size):
    test = initialize(val_feature_path, val_label_path)
    test_images = os.listdir(val_feature_path)
    
    feature = tf.placeholder(tf.float32, shape=[None, image_size, image_size, 3], name='feature')
    label = tf.placeholder(tf.float32, shape=[None], name='label')
    one_hot_label = tf.one_hot(indices=tf.cast(label, tf.int32), depth=80)
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    is_training = tf.placeholder(tf.bool, name='is_training')
    
    logits, loss = conv_network(feature, label, num_class, image_size, keep_prob, is_training)
    values, indices = tf.nn.top_k(logits, 3)

    with tf.Session() as sess:
        saver = tf.train.Saver()
        ckpt = tf.train.get_checkpoint_state(checkpoint_path)
        if ckpt and ckpt.model_checkpoint_path:
            print("Restore the model from checkpoint %s" % ckpt.model_checkpoint_path)
            saver.restore(sess, ckpt.model_checkpoint_path)
            start_step = int(ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1])
        else:
            raise Exception('No checkpoint found')
        
        result = []
        for test_image in test_images:
            temp_dict = {}
            x = test.get_image(os.path.join(val_feature_path, test_image), image_size)
            predictions = np.squeeze(sess.run(indices, feed_dict={feature: np.expand_dims(x, axis=0), keep_prob: 1}), axis=0)
            temp_dict['image_id'] = test_image
            temp_dict['label_id'] = predictions.tolist()
            result.append(temp_dict)
            print('image %s is %d, %d, %d' % (test_image, predictions[0], predictions[1], predictions[2]))
            
        with open('submit.json', 'w') as f:
            json.dump(result, f)
            print('Write result json file, num is %d' % len(result))

In [10]:
val_feature_path = r'E:\ai_challenger\scene classification\dataset\ai_challenger_scene_validation_20170908\scene_validation_images_20170908'
val_label_path = r'E:\ai_challenger\scene classification\dataset\ai_challenger_scene_validation_20170908\scene_validation_annotations_20170908.json'
checkpoint_dir = './checkpoint/'
num_class = 80
image_size = 64

test(val_feature_path, val_label_path, checkpoint_dir, num_class, image_size)

Restore the model from checkpoint ./checkpoint/model.ckpt-59000
INFO:tensorflow:Restoring parameters from ./checkpoint/model.ckpt-59000


NotFoundError: Key conv2d_21/bias not found in checkpoint
	 [[Node: save_1/RestoreV2_270 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/cpu:0"](_recv_save_1/Const_0, save_1/RestoreV2_270/tensor_names, save_1/RestoreV2_270/shape_and_slices)]]
	 [[Node: save_1/RestoreV2_279/_97 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/gpu:0", send_device="/job:localhost/replica:0/task:0/cpu:0", send_device_incarnation=1, tensor_name="edge_1599_save_1/RestoreV2_279", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/gpu:0"]()]]

Caused by op 'save_1/RestoreV2_270', defined at:
  File "D:\Work\Anaconda\envs\scene\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "D:\Work\Anaconda\envs\scene\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "D:\Work\Anaconda\envs\scene\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "D:\Work\Anaco

<a id='step6'></a>
## Step 6: Calculate accuracy.